# EduFlow — Smart Student Productivity Agent  
Multi-agent system for PDF summarization, assignment tracking, daily planning, and study search.

Install & imports

In [26]:
!pip install PyPDF2 beautifulsoup4 requests --quiet
print("Dependencies installed.")

Dependencies installed.


In [27]:
import os
import json
import requests
from bs4 import BeautifulSoup
from PyPDF2 import PdfReader
from datetime import datetime
print("All required packages installed.")

All required packages installed.


Memory Bank

In [28]:
MEMORY_BANK = {"assignments": {}}

Observability Logger

In [29]:
def log_event(agent, event, details=""):
    print({
        "timestamp": datetime.now().isoformat(),
        "agent": agent,
        "event": event,
        "details": details
    })

Assignment Tracker Agent

PDF Text Extractor

In [30]:
def extract_text_from_pdf(path):
    reader = PdfReader(path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    log_event("NotesAgent", "PDF_EXTRACT", path)
    return text

Chunk Text

In [31]:
def chunk_text(text, chunk_size=1200):
    words = text.split()
    chunks, current = [], []
    count = 0

    for w in words:
        current.append(w)
        count += len(w)
        if count > chunk_size:
            chunks.append(" ".join(current))
            current, count = [], 0

    if current:
        chunks.append(" ".join(current))

    return chunks

Fake Summarizer (LLM Stub)

In [32]:
def fake_llm_summarize(chunk):
    log_event("NotesAgent", "SUMMARIZE_CHUNK", f"{len(chunk)} chars")
    return "Summary: " + chunk[:200] + "..."

PDF Summarizer Agent

In [33]:
def summarize_pdf(pdf_path):
    raw = extract_text_from_pdf(pdf_path)
    chunks = chunk_text(raw)

    summaries = []
    for c in chunks:
        summaries.append(fake_llm_summarize(c))

    final_summary = "\n\n".join(summaries)
    log_event("NotesAgent", "FINAL_SUMMARY", f"{len(summaries)} chunks processed")
    return final_summary

Web Search Agent

In [34]:
def simple_web_search(query):
    url = "https://html.duckduckgo.com/html/"
    params = {"q": query}
    res = requests.post(url, data=params)

    soup = BeautifulSoup(res.text, "html.parser")
    results = []

    for a in soup.select(".result__a")[:5]:
        results.append({"title": a.text, "link": a["href"]})

    log_event("SearchAgent", "SEARCH", query)
    return results

Save State

In [35]:
with open("eduflow_demo_state.json", "w") as f:
    json.dump(MEMORY_BANK, f, indent=2)

log_event("System", "STATE_SAVED", "eduflow_demo_state.json created")

{'timestamp': '2025-11-20T16:16:15.658994', 'agent': 'System', 'event': 'STATE_SAVED', 'details': 'eduflow_demo_state.json created'}


DEMO: Add Assignments

In [36]:
timetable = {
    "2025-11-25": [
        {"subject": "Maths", "time": "9:00 AM"},
        {"subject": "Physics", "time": "11:00 AM"},
    ]
}

plan = create_daily_plan("emil", "2025-11-25", timetable)
plan

{'timestamp': '2025-11-20T16:16:15.676090', 'agent': 'TrackerAgent', 'event': 'FETCH_PENDING', 'details': '0 tasks'}
{'timestamp': '2025-11-20T16:16:15.676461', 'agent': 'PlannerAgent', 'event': 'GENERATE_PLAN', 'details': '2025-11-25'}


{'date': '2025-11-25',
 'classes': [{'subject': 'Maths', 'time': '9:00 AM'},
  {'subject': 'Physics', 'time': '11:00 AM'}],
 'study_blocks': []}

DEMO: Upload & Summarize PDF

In [37]:
summary = summarize_pdf("/kaggle/input/sample-notes-pdf/Module 4-Part 1.pdf")
print(summary)

{'timestamp': '2025-11-20T16:16:15.949560', 'agent': 'NotesAgent', 'event': 'PDF_EXTRACT', 'details': '/kaggle/input/sample-notes-pdf/Module 4-Part 1.pdf'}
{'timestamp': '2025-11-20T16:16:15.949857', 'agent': 'NotesAgent', 'event': 'SUMMARIZE_CHUNK', 'details': '1402 chars'}
{'timestamp': '2025-11-20T16:16:15.949880', 'agent': 'NotesAgent', 'event': 'SUMMARIZE_CHUNK', 'details': '1400 chars'}
{'timestamp': '2025-11-20T16:16:15.949890', 'agent': 'NotesAgent', 'event': 'SUMMARIZE_CHUNK', 'details': '994 chars'}
{'timestamp': '2025-11-20T16:16:15.949902', 'agent': 'NotesAgent', 'event': 'FINAL_SUMMARY', 'details': '3 chunks processed'}
Summary: EVOLUTION AND GROWTH OF ELECTRONICS EVOLUTION OF ELECTRONICS 1890•First exp. on generation of electromagnetic waves 1894•Sir J C Bose discovered the propagation of radio waves. •Marconi postulated the...

Summary: •All three of them were honoured by the Nobel Prize in Physics in 1956 –that was the first Nobel Prize in engineering device.1947 •First

DEMO: Web Search

In [38]:
simple_web_search("introduction to cyber security notes")

{'timestamp': '2025-11-20T16:16:16.159103', 'agent': 'SearchAgent', 'event': 'SEARCH', 'details': 'introduction to cyber security notes'}


[]